<h1>Clasificación de falla con redes neuronales</h1>

<h2>Trabajo con digsilent para generar los registros de fallas</h2>

In [1]:
import sys
sys.path.append(r"C:\\Program Files\\DIgSILENT\\PowerFactory 2021 SP7\\Python\\3.9")
import powerfactory as pf
import pandas as pd
import os
import numpy as np

In [2]:
app=pf.GetApplicationExt()
user=app.GetCurrentUser()
print(user)

<l3>\56965.IntUser</l3>


In [3]:
app.ActivateProject('\\56965\\simulacion_comtrade.IntPrj')
proyecto=app.GetActiveProject()
print(proyecto)

<l3>\56965.IntUser\simulacion_comtrade.IntPrj</l3>


In [4]:
carpeta_estudio=proyecto.GetContents("Study Cases.IntPrjfolder")[0]

In [5]:
caso_estudio=carpeta_estudio.GetContents("Study Case")[0]

In [6]:
caso_estudio.Activate()

1

In [7]:
#comando de condiciones iniciales
com_init = app.GetFromStudyCase('ComInc')

#comando ejecutar simulaciones EMT
com_run = app.GetFromStudyCase('ComSim')    

#objeto de eventos
eventos = caso_estudio.GetContents('eventos.IntEvt')[0]

#objeto para exportar resultados
export_resul = caso_estudio.GetContents('Result Export.ComRes')[0]

In [8]:
for eve in eventos.GetContents():
    eve.Delete()

In [9]:
#evento tipo cortocircuito
evtCC = eventos.CreateObject('EvtShc', 'cortocircuito')
#evento tipo despeje de falla
evtDes = eventos.CreateObject('EvtShc', 'despejar')
#evento tipo apertura interruptores
evtOpen = eventos.CreateObject('EvtSwitch', 'abrir')

In [10]:
linea1 = user.SearchObject(r'\56965.IntUser\simulacion_comtrade.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\linea_1.ElmLne')
linea2 = user.SearchObject(r'\56965.IntUser\simulacion_comtrade.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\linea_2.ElmLne')
linea3 = user.SearchObject(r'\56965.IntUser\simulacion_comtrade.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\linea_3.ElmLne')

In [11]:
linea3

<powerfactory.DataObject <l1>\56965.IntUser\simulacion_comtrade.IntPrj\Network Model.IntPrjfolder\Network Data.IntPrjfolder\Grid.ElmNet\linea_3.ElmLne</l1>>

In [12]:
#Variables de TTMM
varTC=['s:I2r_A','s:I2r_B','s:I2r_C']
varTP=['s:U2r_A','s:U2r_B','s:U2r_C']

In [13]:
tp1 = linea1.GetCubicle(0).GetContents('TTPP_200000_100.StaVt')[0] 
tc1 = linea1.GetCubicle(0).GetContents('TTCC_500_5.StaCt')[0]

tp2 = linea1.GetCubicle(1).GetContents('TTPP_200000_100.StaVt')[0] 
tc2 = linea1.GetCubicle(1).GetContents('TTCC_300_1.StaCt')[0]

In [14]:
#crear objeto resultado
#resultado = stCase.CreateObject('ElmRes', 'Resultado')
resultado = caso_estudio.GetContents('Resultado.ElmRes')[0]

if resultado != None:
    resultado.Delete()
    resultado = caso_estudio.CreateObject('ElmRes', 'Resultado')

for tp in [tp1, tp2]: 
    for v in varTP:
        resultado.AddVariable(tp,str(v))
        
for tc in [tc1, tc2]: 
    for v in varTC:
        resultado.AddVariable(tc,str(v))

In [15]:
#Se definen los tiempos de simulacion, inicio, despeje y apertura de interruptores
t_ini = 0.3
t_despeje = 0.8
t_abrir = 0.8
t_stop = 1

#Se hacen ajustes de los eventos

#Distancia de linea, este parametro habilitaba en las versiones anteriores de digsilent la opcion
#de simular fallas en la linea en EMT
#linea1.ishclne = 1
#linea2.ishclne = 1
#linea3.ishclne = 1

#Se ajustan los parametros del evento de cortocircuito
#evtCC.p_target = linea
evtCC.htime = 0
evtCC.mtime = 0
evtCC.time = t_ini
evtCC.X_f = 0

#Se ajustan los parametros del evento de despeje de falla
evtDes.i_shc = 4
#evtDes.p_target = linea
evtDes.time = t_despeje
evtDes.htime = 0
evtDes.mtime = 0
evtDes.i_clearShc = 0

#Se ajustan los parametros del evento de apertura de falla
#evtOpen.p_target = linea
evtOpen.htime = 0
evtOpen.mtime = 0
evtOpen.time = t_abrir
evtOpen.i_switch = 0
evtOpen.i_allph = 1

In [16]:
#se ajustan los parametros del comando de condiciones iniciales
com_init.iopt_sim = 'ins' #tipo de simulación EMT
com_init.p_resvar = resultado
com_init.p_event = eventos

#se ajusta el tiempo de simulación en el comando de ejecución de la simulacion EMT
com_run.tstop = t_stop

In [17]:
#Se ajustan los parametros del comando de exportancion para generar archivos comtrade
export_resul.dSampling = 1000
export_resul.iopt_csel = 1
export_resul.pResult = resultado
export_resul.iopt_exp = 3

In [18]:
#Se definen las caracteristicas de las fallas que se simularan sobre la linea
#tipoFalla = {0:'3f',1:'2f', 2:'1f', 3:'2fg'}
#tipoFalla = {2:'1f'}
#resistencia = list(range(0,105,5))
#distancia = list(range(1,100,1))

#tipoFalla = {0:'3f'}
#resistencia = [0, 5, 10, 25, 50]
#distancia = [50]

In [19]:
#ruta_resultados = 'C:\\Users\\56965\\Documents\\Master_España\\TFM\\resultados2\\'
ruta_resultados = 'C:\\Users\\56965\\Documents\\python\\Inel_ML\\clases\\modulo10\\tensorflowjs\\resultados\\'

lineas = [linea1, linea2, linea3]

#Se ajustan los parametros resistencia de falla, tipo de falla
evtCC.R_f = 0
evtCC.i_shc = 2

for linea in lineas:
    evtCC.p_target = linea
    evtDes.p_target = linea
    evtOpen.p_target = linea
                
    for i in range(100):
        
        #distancia de falla
        d = np.random.randint(1,100)
        
        #linea.fshcloc = d en este parametro se ajustaba la distancia anteriormente
        evtCC.shcLocation = d

        #Se ejecutan los comandos de condiciones iniciales y de simulacion EMT
        com_init.Execute()
        com_run.Execute()

        #Se ajustan parametros del comando de exportacion y se ejecuta (exportar) para el extremo 0
        export_resul.f_name = f"{ruta_resultados}resultado_{'1f'}_{d}_{linea.loc_name}_E0.dat"
        export_resul.resultobj = [resultado, resultado, resultado, resultado, resultado, resultado]
        export_resul.element = [tp1, tp1, tp1, tc1, tc1, tc1]
        export_resul.cvariable = varTP + varTC
        export_resul.Execute()

        #Se ajustan parametros del comando de exportacion y se ejecuta (exportar) para el extremo 1
        export_resul.f_name = f"{ruta_resultados}resultado_{'1f'}_{d}_{linea.loc_name}_E1.dat"
        export_resul.resultobj = [resultado, resultado, resultado, resultado, resultado, resultado]
        export_resul.element = [tp2, tp2, tp2, tc2, tc2, tc2]
        export_resul.cvariable = varTP + varTC
        export_resul.Execute()

In [45]:
d

17

<h2>Leer archivos de resultados</h2>

In [21]:
ruta_resultados = 'C:\\Users\\56965\\Documents\\python\\Inel_ML\\clases\\modulo10\\tensorflowjs\\resultados\\'

In [22]:
archivo_E0_linea1 = [f.split('.')[0] for f in os.listdir(ruta_resultados) if f.find('_linea_1_E0.cfg')>=0]
archivo_E0_linea3 = [f.split('.')[0] for f in os.listdir(ruta_resultados) if f.find('_linea_3_E0.cfg')>=0]

In [23]:
#extremo E0 contiguo a la linea_3
#extremo E1 contiguo a la linea_2

In [24]:
def leer_comtrade(ruta_archivo_cfg, ruta_archivo_dat):
    
    #se lee el archivo .cfg para extraer los coeficientes de la ecuación a*x+b de la norma del comtrade
    with open(ruta_archivo_cfg, 'r') as f:
        contenido = f.readlines()
        contenido = [x.rstrip('\n') for x in contenido]
    
    n_registros = int(contenido[1].split(',')[0])
    n_registros_A = int(contenido[1].split(',')[1].rstrip('A'))
    n_registros_D = int(contenido[1].split(',')[2].rstrip('D'))
    
    var_analogicas = {c.split(',')[1]:{'unidad': c.split(',')[4], 
                                   'A': float(c.split(',')[5]), 
                                   'B': float(c.split(',')[6]),
                                   'primario': float(c.split(',')[-3]),
                                   'secundario': float(c.split(',')[-2]),
                                   'escala': c.split(',')[-1]
                                  }
                      for c in contenido[2:n_registros_A+2]}
    
    #frecuencia del sistema
    f = float(contenido[n_registros + 2])
    #frecuencia de muestreo
    f_muestreo = float(contenido[n_registros + 4].split(',')[0])
    
    #lectura archivo .dat
    archivo_dat = np.genfromtxt(ruta_archivo_dat, delimiter=',')
    
    for i,(k,v) in enumerate(var_analogicas.items()):
        fanalogv = np.vectorize(lambda x: v['A'] * x + v['B'])
        archivo_dat[:,i+2] = fanalogv(archivo_dat[:,i+2])

    #se agregan archivo de caracteristicas de entrada X
    return (archivo_dat[:, 2:n_registros + 2], var_analogicas, f, f_muestreo)

In [25]:
valores, analogicas, f, f_muestreo = leer_comtrade(ruta_resultados+archivo_E0_linea1[0]+'.cfg', ruta_resultados+archivo_E0_linea1[0]+'.dat')

In [26]:
#comtrade medidos en el extremo E0 para fallas dentro de la linea 1
data_E0_linea1 = np.array([leer_comtrade(ruta_resultados+archivo+'.cfg', ruta_resultados+archivo+'.dat')[0] 
                              for archivo in archivo_E0_linea1])

In [27]:
data_E0_linea1.shape

(67, 1101, 6)

In [28]:
#comtrade medidos en el extremo E0 para fallas fuera de la linea 1 (en a linea 3)
data_E0_linea3 = np.array([leer_comtrade(ruta_resultados+archivo+'.cfg', ruta_resultados+archivo+'.dat')[0] 
                              for archivo in archivo_E0_linea3])

In [29]:
data_E0_linea3.shape

(62, 1101, 6)

In [30]:
data_x = np.concatenate((data_E0_linea1, data_E0_linea3))

In [34]:
data_y = np.array([1 if x <=67 else 0 for x in range(67+62)])

In [37]:
data_y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [35]:
data_y.shape

(129,)

In [36]:
data_x.shape

(129, 1101, 6)

<h2>Construir Red Neuronal</h2>

In [38]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder = LabelEncoder()
encoder.fit(data_y)
LabelEncoder()

data_y2 = encoder.transform(data_y)

data_y2 = np_utils.to_categorical(data_y2)

In [39]:
data_y2.shape

(129, 2)

In [58]:
data_y2

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam

In [42]:
model = Sequential()
model.add(LSTM(100, input_shape=(1101, 6)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [43]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [44]:
history = model.fit(data_x, data_y2, epochs=10, batch_size=64, verbose=2)

Epoch 1/10
3/3 - 22s - loss: 0.7079 - accuracy: 0.5349 - 22s/epoch - 7s/step
Epoch 2/10
3/3 - 15s - loss: 0.5602 - accuracy: 0.8062 - 15s/epoch - 5s/step
Epoch 3/10
3/3 - 15s - loss: 0.4185 - accuracy: 0.9147 - 15s/epoch - 5s/step
Epoch 4/10
3/3 - 17s - loss: 0.3827 - accuracy: 0.9690 - 17s/epoch - 6s/step
Epoch 5/10
3/3 - 15s - loss: 0.3190 - accuracy: 0.9767 - 15s/epoch - 5s/step
Epoch 6/10
3/3 - 15s - loss: 0.2685 - accuracy: 0.9767 - 15s/epoch - 5s/step
Epoch 7/10
3/3 - 15s - loss: 0.2207 - accuracy: 0.9845 - 15s/epoch - 5s/step
Epoch 8/10
3/3 - 17s - loss: 0.1849 - accuracy: 0.9922 - 17s/epoch - 6s/step
Epoch 9/10
3/3 - 17s - loss: 0.1752 - accuracy: 0.9767 - 17s/epoch - 6s/step
Epoch 10/10
3/3 - 19s - loss: 0.1536 - accuracy: 0.9922 - 19s/epoch - 6s/step


In [46]:
prediccion = [np.argmax(v) for v in model.predict(data_x)]

5/5 [==============================] - 4s 475ms/step


In [47]:
prediccion

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [48]:
model.save("model_comtrade.h5")

<h2>Exportar modelo a tensoflowjs</h2>

In [ ]:
#!pip install tensorflowjs

In [49]:
!tensorflowjs_converter --input_format keras "model_comtrade.h5" ./jsmodel

<h2>Crear archivos de prueba</h2>

In [50]:
archivo_interno = leer_comtrade(ruta_resultados+archivo_E0_linea1[0]+'.cfg', ruta_resultados+archivo_E0_linea1[0]+'.dat')[0]

In [51]:
archivo_externo = leer_comtrade(ruta_resultados+archivo_E0_linea3[0]+'.cfg', ruta_resultados+archivo_E0_linea3[0]+'.dat')[0]

In [52]:
np.savetxt("interno.csv", archivo_interno, delimiter=",")
np.savetxt("externo.csv", archivo_externo, delimiter=",")

In [54]:
archivo_interno

(1101, 6)

In [56]:
model.predict(archivo_interno.reshape(1,1101,6))

1/1 [==============================] - 0s 132ms/step


array([[0.10055943, 0.8994405 ]], dtype=float32)

In [57]:
model.predict(archivo_externo.reshape(1,1101,6))

1/1 [==============================] - 0s 187ms/step


array([[0.9931251 , 0.00687484]], dtype=float32)